In [3]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder  # Assuming you have this util function
print('doing something')
# === CONFIG ===
tpafile = './databases/TPAtable.csv'
tablesfolder = 'countriestables'
satnamesfolder = 'satellitenames'
outfolder = 'adm_conflicts'
countrieslistfile = 'countrieslist.csv'

# Create output folder if needed
if not os.path.exists(outfolder):
    os.makedirs(outfolder)

# Load country codes
with open(countrieslistfile, 'r') as f:
    countries = f.read().strip().split(', ')
# # comment this for custom countries list
countries = ['CAN'] 

# Load reference table
refdf = pd.read_csv(tpafile)

# === PROCESS EACH COUNTRY ===
for ccode in countries:
    print(f"\n=== Processing {ccode} ===")

    # Make folder for this country
    country_outfolder = os.path.join(outfolder, ccode)
    os.makedirs(country_outfolder, exist_ok=True)

    # --- Step 1: Find conflicts ---
    noinfo = country_conflicts_finder(ccode, refdf, tablesfolder, satnamesfolder, country_outfolder)
    if not noinfo:
        continue
    # Save list of satellites with missing carrier frequency info
    with open(os.path.join(country_outfolder, 'noinfooncarrierfrequency.txt'), 'w') as f:
        f.write('The following list contains all the satellite names for which no information on the carrier frequency was found in the ITU database for at least one of the entries.\n')
        f.write('It is possible that the same satellite has other entries with all the required information for channel overlap comparisons.\n\n')
        f.write(str(noinfo))

    # --- Step 2: Create summary table ---
    folder = os.path.join(country_outfolder, 'output_tables')
    all_files = os.listdir(folder)
    worstcase_files = [f for f in all_files if 'worstcase' in f.lower() and 'R_E' not in f]

    fmins, fmaxs, f0s, bws, types, dataframes, satnames_lists = [], [], [], [], [], [], []

    for ff in worstcase_files:
        match = re.search(r'(\d+\.\d+)-(\d+\.\d+)_([A-Z]_[A-Z])', ff)
        if match:
            fmin = float(match.group(1))
            fmax = float(match.group(2))
            f0 = (fmin + fmax) / 2
            bw = round(1e6 * (fmax - fmin))
            ctype = str(match.group(3))
            df = pd.read_csv(os.path.join(folder, ff), low_memory=False)
            satnames = df[' com_el.sat_name'].unique()

            fmins.append(fmin)
            fmaxs.append(fmax)
            f0s.append(f0)
            bws.append(bw)
            types.append(ctype)
            dataframes.append(df)
            satnames_lists.append(satnames)

    summary = []
    for f0_, bw_, ctype, sats in zip(f0s, bws, types, satnames_lists):
        row = {
            'f0': f0_,
            'bandwidth': bw_,
            'conflict_type': ctype,
            'satellite_names': ', '.join(sats),
            'satellite_count': len(sats)
        }
        summary.append(row)

    df_summary = pd.DataFrame(summary)
    df_summary.to_csv(os.path.join(country_outfolder, 'conflict_summary_by_freq_type.csv'), index=False)

    # --- Step 3: Plot summary ---
    records = []
    for f0_, bw_, ctype, sats in zip(f0s, bws, types, satnames_lists):
        if ctype == 'R_E':
            continue
        label = f"{f0_:.3f} MHz / {bw_//1000} kHz"
        records.append({
            'freq_band': label,
            'conflict_type': ctype,
            'satellite_count': len(sats)
        })

    df_plot = pd.DataFrame(records)

    if not df_plot.empty:
        pivot = df_plot.pivot_table(
            index='freq_band',
            columns='conflict_type',
            values='satellite_count',
            aggfunc='sum',
            fill_value=0
        )

        ax = pivot.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='tab20')
        plt.ylabel('Number of satellites')
        plt.xlabel('Frequency / Bandwidth')
        plt.title(f'Conflict types by frequency band for {ccode} (excluding R_E)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        plot_path = os.path.join(country_outfolder, 'conflict_type_barplot.png')
        plt.savefig(plot_path, dpi=300)
        plt.close()
        print(f"✅ Plot saved to: {plot_path}")

    # --- Step 4: Create pivot table (satellite vs worst case percent) ---
    sat_conflict_map = defaultdict(lambda: defaultdict(int))

    for ff, fmin, fmax, f0, bw, conflict_type, satnames, df in zip(worstcase_files, fmins, fmaxs, f0s, bws, types, satnames_lists, dataframes):
        col_prefix = ff.split('_')[0]
        col_suffix = conflict_type[0]
        conflict_column = f"{col_prefix}_{col_suffix}"

        if conflict_column not in df.columns:
            print(f"⚠️ Column {conflict_column} not found in {ff}")
            continue

        for satname in satnames:
            satname = satname.strip()
            try:
                val = df[df[' com_el.sat_name'] == satname][conflict_column].values[0]
            except IndexError:
                continue

            if isinstance(val, str) and '%' in val:
                try:
                    percent = int(re.search(r'(\d+)', val).group(1))
                    conflict_label = f"{f0:.3f}_{bw//1000}kHz_{conflict_type}"
                    sat_conflict_map[satname][conflict_label] = max(
                        sat_conflict_map[satname][conflict_label], percent
                    )
                except Exception as e:
                    print(f"⚠️ Could not parse value '{val}' for satellite {satname}: {e}")

    records = []
    for satname, conflicts in sat_conflict_map.items():
        for conflict_label, percent in conflicts.items():
            records.append({
                'sat_name': satname,
                'conflict_type': conflict_label,
                'worst_case_percent': percent
            })

    df_pivot_source = pd.DataFrame(records)

    if df_pivot_source.empty:
        print("⚠️ No data to create pivot table.")
    else:
        pivot_table = df_pivot_source.pivot(
            index='sat_name',
            columns='conflict_type',
            values='worst_case_percent'
        )
        pivot_path = os.path.join(country_outfolder, 'conflict_percent_pivot_table.csv')
        pivot_table.to_csv(pivot_path)
        print(f"✅ Pivot table saved to: {pivot_path}")

print("\n🎯 Done with all countries!")


doing something

=== Processing CAN ===
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
loading  CAN.csv
finding conflicts for  CAN
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


file saved to  adm_conflicts\CAN\conflicts.csv
satellite names for CAN  : 
 ['ADS', 'BRITE-CANADA', 'CANSAT-LEO-S2', 'CANX-2', 'CANX-4-5', 'CASSIOPE-1A', 'CHORUS-C1', 'EV10', 'GHGSAT-CX', 'GHGSAT-D', 'GJP', 'HOTH', 'KELYPSIS', 'M3MSAT', 'NEOSSAT-1A', 'QEYSSAT', 'RADARSAT-2C', 'RADARSAT-3C', 'SAPPHIRE', 'SCISAT-1']


C:\Users\smen851\PycharmProjects\TPA1_ITU_SE_SIEVE\ITUtils.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf[['tpaconflicts', 'percentoverlap']] = ddf.parallel_apply(
C:\Users\smen851\PycharmProjects\TPA1_ITU_SE_SIEVE\ITUtils.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_with_nan['tpaconflicts'] = ''
C:\Users\smen851\PycharmProjects\TPA1_ITU_SE_SIEVE\ITUtils.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,...,carrier_fr.freq_carr,carrier_fr.seq_emiss,carrier_fr.seq_no,srv_area.ctry,srv_area.f_excl_api,channel.bandwidth,channel.freq_min,channel.freq_max,tpaconflicts,percentoverlap
0,101500360,NaN,CAN,NaN,SCISAT-1,NaN,11.2,15/11/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,6000000.0,NaN,NaN,,
1,101500360,NaN,CAN,NaN,SCISAT-1,NaN,11.2,15/11/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,6000000.0,NaN,NaN,,
2,101500360,NaN,CAN,NaN,SCISAT-1,NaN,11.2,15/11/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,2300000.0,NaN,NaN,,
3,101500360,NaN,CAN,NaN,SCISAT-1,NaN,11.2,15/11/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,1100000.0,NaN,NaN,,
4,101500360,NaN,CAN,NaN,SCISAT-1,NaN,11.2,15/11/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,2300000.0,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,122500237,NaN,CAN,NaN,GHGSAT-CX,NaN,11.2,28/12/2022 12:00:00 am,50,6,...,NaN,NaN,NaN,NaN,NaN,92000.0,NaN,NaN,,
1384,122500237,NaN,CAN,NaN,GHGSAT-CX,NaN,11.2,28/12/2022 12:00:00 am,50,7,...,NaN,NaN,NaN,NaN,NaN,92000.0,NaN,NaN,,
1385,122500237,NaN,CAN,NaN,GHGSAT-CX,NaN,11.2,28/12/2022 12:00:00 am,50,8,...,NaN,NaN,NaN,NaN,NaN,92000.0,NaN,NaN,,
1386,122500237,NaN,CAN,NaN,GHGSAT-CX,NaN,11.2,28/12/2022 12:00:00 am,50,9,...,NaN,NaN,NaN,NaN,NaN,92000.0,NaN,NaN,,


names with no frequency info ['SCISAT-1', 'RADARSAT-2C', 'CANX-2', 'ADS', 'CASSIOPE-1A', 'M3MSAT', 'SAPPHIRE', 'NEOSSAT-1A', 'CANX-4-5', 'GHGSAT-CX']
expanding the conflict columns for CAN
['TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R', 'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E', 'TPA1.401.89525-401.90475_E', 'TPA1.2202.4-2203.4_E']


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
['TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R', 'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E', 'TPA1.401.89525-401.90475_E', 'TPA1.2202.4-2203.4_E']
table saved to adm_conflicts\CAN\output_tables\TPA1.2055.475-2055.725_R_R.csv
table saved to adm_conflicts\CAN\output_tables\TPA1.2055.475-2055.725_R_R_worstcase.csv
table saved to adm_conflicts\CAN\output_tables\TPA1.2237.0-2238.0_E_E.csv
table saved to adm_conflicts\CAN\output_tables\TPA1.2237.0-2238.0_E_E_worstcase.csv
table saved to adm_conflicts\CAN\output_tables\TPA1.2065.575-2065.825_R_R.csv
table saved to adm_conflicts\CAN\ou